In [1]:
!pip install datasets
!pip install soundfile
!pip install librosa


{
    "tags": [
        "hide-output"
    ]
}

{'tags': ['hide-output']}

In [ ]:
import datasets
import json
from datasets import load_dataset
cv_11 = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split='invalidated')
cv_11 = cv_11.sort('client_id')
cv_11 = cv_11.select([j for j in range(10000)])
cv_11 = cv_11.remove_columns(['path', 'sentence', 'up_votes', 'down_votes', 'age', 'accent', 'locale', 'segment'])

In [ ]:
from pathlib import Path


In [3]:
import numpy as np
import numpy.random as rnd
import random as rand
rnd.seed(0)
rand.seed(0)

In [ ]:

import os

import soundfile as sf
import time
bande_organise = {}

for x in cv_11:
    if (x['gender'] == 'male'):
        keyx = x['client_id'] + 'M'
    else:
        keyx = x['client_id'] + 'F'
    if  keyx not in bande_organise.keys():
        bande_organise[keyx] = [x['audio']]
    else:
        bande_organise[keyx].append(x['audio'])

In [ ]:
L =  list(bande_organise.keys())

In [ ]:
main_speaker = {}
interruptor = {}
for key in L:
    if len(bande_organise[key])>30:
        main_speaker[key] = bande_organise[key]
    elif len(bande_organise[key])>4 and len(bande_organise[key])<10:
        interruptor[key] = bande_organise[key]
L = list(main_speaker.keys())
m = min([len(main_speaker[key]) for key in L])
Lint = list(interruptor.keys())
l = len(L)
lint = len(Lint)

In [39]:
cv_11[0]['audio']

{'path': '/Users/timou/.cache/huggingface/datasets/downloads/extracted/5c5cedf3750d077d8aec7bdba2f0a47a4b17d92e8e73992d4f0d3572e70887e1/fr_invalidated_0/common_voice_fr_19712064.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.62077646e-10, 4.75310291e-10, 1.19993529e-10]),
 'sampling_rate': 48000}

In [37]:
start_time=time.time()
N = 10
out= Path.home() /'outputs'
max_interruptions = 5
for i in range(N):
    flag = True
    speaker_1 = int(rnd.uniform(0,l))
    speaker_2 = int(rnd.uniform(0,lint))
    N_interruptions = int(rnd.uniform(1,max_interruptions))
    interruptions = rand.sample(range(m), N_interruptions)
    L_aud = []
    positions = []
    t = 0
    path = str(speaker_1)+L[speaker_1][-1] +"-"+ str(speaker_2)+Lint[speaker_2][-1] + "_"
    for k in range(m):
        inter = 0
        if k not in interruptions:
            piste = main_speaker[L[speaker_1]][k]
            audio_data= piste['array']
            if (int(piste['sampling_rate']) != 48000):
                flag =False
            L_aud.append(audio_data)
            path = path + "0"
        else:
            inter = inter+1
            piste = interruptor[Lint[speaker_2]][inter]
            audio_data= piste['array']
            if (int(piste['sampling_rate']) != 48000):
                flag =False
            L_aud.append(audio_data)
            path = path +"1"
            positions.append((int(t), int(t+len(audio_data)/float(48000)),Lint[speaker_2][-1], 'N'))
        t += len(audio_data)/float(48000)
    merged_audio = np.concatenate(L_aud, axis=0)
    if flag: 
        pathaudio  = path + ".wav"
        pathjson = path + '.json'
        sf.write(out/pathaudio, merged_audio, 48000)
        #Creating JSON file in ./outputs/
        to_JSON = {'path': (str(out/pathaudio), '-'), 'gender':L[speaker_1][-1],  'start': 0, 'end': int( len(merged_audio)/float(48000)), 'num_interruptions':N_interruptions, 'interruptions': positions  }
        json_d = json.dumps(to_JSON, indent = 2)
        with open(out/pathjson, 'w') as f:
            f.write(json_d)
    else: print('flaaag')    
end_time = time.time()
print(end_time-start_time)

0.567828893661499


In [38]:
python_objects = []
for json_file in out.iterdir():
        try:
            with open(json_file, "r") as f:
                python_objects.append(json.load(f))
            # Dump all the Python objects into a single JSON file.
        except:
            1 == 1
with open('database.json', "w") as f:
    json.dump(python_objects, f, indent=4)
print('Stored in where ipynb file is.')

Stored in where ipynb file is.
